In [ ]:
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# You can adjust the slider any time

dim = widgets.IntSlider(min=2, max=5, value=3, step=1, description="n²×n² Sudoku")
dim

In [ ]:
from local.sudoku import Board

secret_solution = Board.random(dim.value).solve()
public_puzzle = secret_solution.to_puzzle()

print(public_puzzle)

In [ ]:
from local.ec.static import CurvePoint, Scalar, ONE_POINT
from local.ec.util import Opening
from local.graph import Mapping
from typing import List, Tuple
import random

punto_uno, = CurvePoint.sample_greater_one(1)


class Peggy:
    def __init__(self, secret_solution: Board, public_puzzle: Board):
        """
        Give Peggy her solution and the public puzzle.
        """
        self.solution = secret_solution
        self.presets = public_puzzle
        self.dim = self.solution.dim
        self.dim_sq = self.solution.dim_sq
        
    def shuffle(self) -> List[List[CurvePoint]]:
        """
        Peggy shuffles how she displays each value.
        
        She commits to the shuffled value in each cell of her solution.
        
        She sends the commitments to Victor.
        """
        shuffling = Mapping.shuffle_list(list(range(1, self.dim_sq + 1)))
        self.openings = [[Opening(Scalar(shuffling[self.solution[row][col]]), ONE_POINT, punto_uno)
                          for col in range(self.dim_sq)] for row in range(self.dim_sq)]
        commitments = [[opening.close() for opening in row_openings] for row_openings in self.openings]
        return commitments

    def reveal(self, row: int, col: int, mode: str) -> List[Opening]:
        """
        Victor asks Peggy to reveal a given area (row, column, mode).
        
        Peggy returns the openings of this area.
        """
        if mode == "row":
            return [self.openings[row][col] for col in range(self.dim_sq)]
        elif mode == "column":
            return [self.openings[row][col] for row in range(self.dim_sq)]
        elif mode == "box":
            return [self.openings[row + row_offset][col + col_offset]
                    for row_offset in range(self.dim) for col_offset in range(self.dim)]
        else:  # mode == "presets"
            return [self.openings[row][col]
                    for row in range(self.dim_sq) for col in range(self.dim_sq) if self.presets[row][col] > 0]

class Victor:
    def __init__(self, public_puzzle: Board):
        """
        Give Victor the public puzzle.
        
        He does not know the solution.
        """
        self.presets = public_puzzle
        self.dim = self.presets.dim
        self.dim_sq = self.presets.dim_sq
        
    def select(self, commitments: List[List[CurvePoint]]) -> Tuple[int, int, str]:
        """
        Victor receives the commitments from Peggy.
        
        He selects a random area on the board (row, column, mode).
        
        He challenges Peggy to reveal this area.
        """
        self.commitments = commitments
        self.mode = random.choice(["row", "column", "box", "presets"])
        
        if self.mode == "row":
            self.row = random.randrange(self.dim_sq)
            self.col = None
        elif self.mode == "column":
            self.row = None
            self.col = random.randrange(self.dim_sq)
        elif self.mode == "box":
            self.row = random.randrange(0, self.dim_sq, self.dim)
            self.col = random.randrange(0, self.dim_sq, self.dim)
        else:  # self.mode == "presets"
            self.row = None
            self.col = None
        
        return self.row, self.col, self.mode
    
    def verify(self, revealed: List[Opening]) -> bool:
        """
        Victor receives the openings to the area that he selected.
        
        He checks if the area has the correct values:
        
        1. No duplicate or zero values in a row, column or box
        2. Consistent mapping of presets
           (Cells that were equal in presets are equal in the mapped presets, and vice versa for unequal cells)
           
        He checks if the openings match the commitments that Peggy sent.
        
        If everything checks out, he accepts. Otherwise he rejects.
        """
        shuffled_values = [opening.value().n for opening in revealed]
        
        if self.mode == "row":
            commitments = [self.commitments[self.row][col] for col in range(self.dim_sq)]
            if not self.presets.verify_area(shuffled_values):
                return False
        elif self.mode == "column":
            commitments = [self.commitments[row][self.col] for row in range(self.dim_sq)]
            if not self.presets.verify_area(shuffled_values):
                return False
        elif self.mode == "box":
            commitments = [self.commitments[row + row_offset][col + col_offset]
                           for row_offset in range(self.dim) for col_offset in range(self.dim)]
            if not self.presets.verify_area(shuffled_values):
                return False
        else:  # self.mode == "presets"
            commitments = [self.commitments[row][col]
                           for row in range(self.dim_sq) for col in range(self.dim_sq) if self.presets[row][col] > 0]
            if not self.presets.verify_shuffling(iter(shuffled_values)):
                return False
        
        return Opening.batch_verify(revealed, commitments)

In [ ]:
peggy = Peggy(secret_solution, public_puzzle)
victor = Victor(public_puzzle)

commitments = peggy.shuffle()
print(f"Commitments: {commitments}")
row, col, mode = victor.select(commitments)
print(f"Row: {row}, column: {col}, mode: {mode}")
revealed = peggy.reveal(row, col, mode)
print(f"Revealed: {revealed}")

# Victor is convinced
if victor.verify(revealed):
    # Valid solution (good)
    if True:
        print("Convinced 👌 (expected)")
    # Invalid solution (evil)
    else:
        print("Convinced 👌 (Victor was fooled)")
# Victor is not convinced
else:
    # Valid solution (good)
    if True:
        print("Not convinced... 🤨 (Peggy was dumb)")
    # Invalid solution (evil)
    else:
        print("Not convinced... 🤨 (expected)")

In [ ]:
num_exchanges_complete = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_exchanges_complete

In [ ]:
# Honest case
secret_solution2 = Board.random(dim.value).solve()
public_puzzle2 = secret_solution2.to_puzzle()

honest_peggy = Peggy(secret_solution2, public_puzzle2)
victor = Victor(public_puzzle2)

peggy_success = 0

for _ in range(num_exchanges_complete.value):
    commitments = honest_peggy.shuffle()
    row, col, mode = victor.select(commitments)
    revealed = honest_peggy.reveal(row, col, mode)

    if victor.verify(revealed):
        peggy_success += 1
        
peggy_success_rate = peggy_success / num_exchanges_complete.value * 100

print(f"Running {num_exchanges_complete.value} exchanges")
print(f"Honest Peggy wins {peggy_success_rate:0.2f}% of the time")
print()

assert peggy_success_rate == 100
print("Peggy always wins if she is honest")

In [ ]:
num_exchanges_sound = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_rounds = widgets.IntSlider(min=1, max=15, value=1, step=1, description="#Rounds")

display(num_exchanges_sound)
display(num_rounds)

In [ ]:
# Lying case
secret_solution3 = Board.random(dim.value).solve()
secret_solution3.falsify(1)
public_puzzle3 = secret_solution2.to_puzzle()

lying_peggy = Peggy(secret_solution3, public_puzzle3)
victor = Victor(public_puzzle3)

victor_success = 0

for _ in range(num_exchanges_sound.value):
    for _ in range(num_rounds.value):
        commitments = lying_peggy.shuffle()
        row, col, mode = victor.select(commitments)
        revealed = lying_peggy.reveal(row, col, mode)
    
        if not victor.verify(revealed):
            victor_success += 1
            break
            
victor_success_rate = victor_success / num_exchanges_sound.value * 100

print(f"Running {num_exchanges_sound.value} exchanges with {num_rounds.value} rounds each")
print(f"Victor wins against lying Peggy {victor_success_rate:0.2f}% of the time")
print()

if victor_success_rate < 50:
    print("Victor loses quite often for a small number of rounds")
elif victor_success_rate < 90:
    print("Victor gains more confidence with each added round")
else:
    print("At some point it is basically impossible to fool Victor")

In [ ]:
num_transcripts = widgets.IntSlider(min=1000, max=100000, value=10000, step=1000, description="#Transcripts")
num_transcripts

In [ ]:
from typing import Tuple
import local.stats as stats
from itertools import chain

# Make sure to run this for small grids (adjust the parameters)
# Large grids can lead to errors
# The list of critical chi-square values might be too short for the large number of degrees of freedom
# Large grids also require very many transcripts, which is slow

peggy = Peggy(secret_solution, public_puzzle)
victor = Victor(public_puzzle)

def real_transcript() -> Tuple:
    commitments = peggy.shuffle()
    row, col, mode = victor.select(commitments)
    revealed = peggy.reveal(row, col, mode)

    return CurvePoint.batch_serialize(chain(*commitments), compact=2)[0:3], row, col, mode, \
           Opening.batch_serialize(revealed, compact=2)[0:3]


dim = dim.value
dim_sq = dim ** 2

def fake_transcript() -> Tuple:
    mode = random.choice(["row", "column", "box", "presets"])
    if mode == "row":
        row = random.randrange(dim_sq)
        col = None
        size_revealed = dim_sq
    elif mode == "column":
        row = None
        col = random.randrange(dim_sq)
        size_revealed = dim_sq
    elif smode == "box":
        row = random.randrange(0, dim_sq, dim)
        col = random.randrange(0, dim_sq, dim)
        size_revealed = dim_sq
    else:  # mode == "presets"
        # FIXME: Complicated
        row = None
        col = None
        size_revealed = sum(1 for columns in public_presets for value in columns if value > 0)
    
    commitments = [[CurvePoint.random() for _ in range(dim_sq)] for _ in range(dim_sq)]
    revealed = []
    
    return CurvePoint.batch_serialize(chain(*commitments), compact=2)[0:3], row, col, mode, \
           Opening.batch_serialize(revealed, compact=2)[0:3]

print("Real transcript: {}".format(real_transcript()))
print("Fake transcript: {}".format(fake_transcript()))
print()

real_samples = [real_transcript() for _ in range(num_transcripts.value)]
fake_samples = real_samples

# The chi-square test is only valid if most bins are filled
# Increase the number of transcripts if there are too many empty bins

null_hypothesis = stats.chi_square_equal(real_samples, fake_samples)
print()

if null_hypothesis:
    print("Real and fake transcripts are the same distribution.")
    print("Victor learns nothing 👌")
else:
    print("Real and fake transcripts are different distributions.")
    print("Victor might learn something 😧")

stats.plot_comparison(real_samples, fake_samples, "real", "fake")